In [6]:
import json
import pandas as pd
import pyspark.sql.functions as F
from cafu.metadata import path
path_odds = path('dir_results')+'/odds/campeonato=franca_index=1.json'
r = open(path_odds, 'r')
odds = json.load(r)

In [2]:
pd.set_option('display.max_columns', None)
from cafu.utils import get_spark
spark = get_spark(1)

22/03/19 13:50:39 WARN Utils: Your hostname, luan-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/03/19 13:50:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/luan/.ivy2/cache
The jars for the packages stored in: /home/luan/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/spark/spark-3.0.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-29173468-4ba2-4fb4-a187-eba1662f39fa;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in cen

In [3]:
from cafu.utils import proximas_partidas
from cafu.metadata import campeonato_dafabet
from cafu.queries import GetOdds
from datetime import datetime, date, timedelta
from unidecode import unidecode

In [4]:
# campeonatos que terão jogos, hoje ou amanhã
pr_part = proximas_partidas()
dafabet = campeonato_dafabet()
after_tomorrow = date.today()+timedelta(days=2)
campeonatos = []
for c in pr_part:
    for t in pr_part[c]:
        try:
            prox_jogo = datetime.strptime(pr_part[c][t], '%Y-%m-%d').date()
            if (prox_jogo<=after_tomorrow) and (c in dafabet):
                campeonatos.append([c,t])
        except:
            pass

In [5]:
query = GetOdds()
for c in campeonatos:
    stop = False
    index = 0
    while not stop:
        query.get_campeonato_dafabet(c[0])
        descricao_partida = query.join_link_odds_partida(index)
        if descricao_partida is not None:
            if (
                ('Hoje' in descricao_partida['horario']) or
                ('Amanhã' in descricao_partida['horario'])
               ):
                query.open_odds()
                odds = query.get_odds()
                for k1 in odds:
                    for k2 in odds[k1]:
                        odds[k1][k2] = float(odds[k1][k2])
                for k1 in odds:
                    df = spark.createDataFrame([odds[k1]])
                    df = (
                             df
                             .withColumn('time_casa', F.lit(descricao_partida['time_casa']))
                             .withColumn('time_visitante', F.lit(descricao_partida['time_visitante']))
                             .withColumn('horario', F.lit(descricao_partida['horario']))
                             .withColumn('campeonato_metadata', F.lit(c))
                             .withColumn('temporada_metadata', F.lit(t))
                             .withColumn('date_update', F.lit(datetime.now()))
                         )
                    for c in df.columns:
                        df = df.withColumnRenamed(c, (c.replace(' ','_')
                                                       .replace(',','_'))
                                                 )
                    dir_ = k1.replace('/','-')
                    dir_ = unidecode(dir_.lower())
                    df.write.parquet(f'teste/{dir_}', mode='append')
            else:
                stop = True
        else:
            stop = True
        index+=1
query.web.close() # encerra a sessão

 10%|█         | 1/10 [00:05<00:49,  5.52s/it]


In [9]:
z = odds, 4

In [11]:
len(z[0])

77